In [11]:
from argparse import ArgumentParser
import pandas as pd
import numpy as np
import sys


def topsis(df, weights, impacts):
    data = df.iloc[:, 1:].values.astype(float)

    norm = np.sqrt((data**2).sum(axis=0))
    norm_data = data / norm

    weighted = norm_data * weights

    ideal_best = []
    ideal_worst = []

    for j in range(weighted.shape[1]):
        if impacts[j] == "+":
            ideal_best.append(weighted[:, j].max())
            ideal_worst.append(weighted[:, j].min())
        else:
            ideal_best.append(weighted[:, j].min())
            ideal_worst.append(weighted[:, j].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    d_pos = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    d_neg = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    score = d_neg / (d_pos + d_neg)
    return score


def main() -> None:
    parser = ArgumentParser(description="TOPSIS command-line tool")

    parser.add_argument("input", help="Input CSV file")
    parser.add_argument("weights", help="Comma-separated weights (e.g. 1,2,3)")
    parser.add_argument("impacts", help="Comma-separated impacts (e.g. +,-,+)")
    parser.add_argument("output", help="Output CSV file")

    args = parser.parse_args()

    try:
        if args.input.endswith(".csv"):
            df = pd.read_csv(args.input)
        elif args.input.endswith(".xlsx"):
            df = pd.read_excel(args.input)
        else:
            print("Error: Unsupported file format. Use .csv or .xlsx")
            sys.exit(1)
    except Exception:
        print("Error: Unable to read input file")
        sys.exit(1)

    try:
        weights = np.array(list(map(float, args.weights.split(","))))
        impacts = args.impacts.split(",")
    except Exception:
        print(np.array(list(map(float, args.weights.split(",")))))
        print(args.impacts.split(","))
        print("Error: Invalid weights or impacts format")
        sys.exit(1)

    if len(weights) != len(impacts):
        print("Error: Weights and impacts length mismatch")
        sys.exit(1)

    if df.shape[1] - 1 != len(weights):
        print("Error: Number of weights must match number of criteria")
        sys.exit(1)

    if not all(i in ["+", "-"] for i in impacts):
        print("Error: Impacts must be + or -")
        sys.exit(1)

    scores = topsis(df, weights, impacts)

    df["Topsis Score"] = scores
    df["Rank"] = df["Topsis Score"].rank(ascending=False).astype(int)

    df.to_csv(args.output, index=False)
    print(f"Results saved to {args.output}")


def run_topsis(input_file, weights, impacts, output_file):
    df = pd.read_excel(input_file)
    weights = np.array(weights)
    scores = topsis(df, weights, impacts)
    df["Topsis Score"] = scores
    df["Rank"] = df["Topsis Score"].rank(ascending=False).astype(int)
    df.to_csv(output_file, index=False)
    print(df.head())

# if __name__ == "__main__":
#     main()

In [12]:
run_topsis(
    "data.xlsx",
    [1, 1, 1, 1, 1],
    ["+", "+", "+", "-", "-"],
    "output.csv"
)


  Fund Name    P1    P2   P3    P4     P5  Topsis Score  Rank
0        M1  0.84  0.71  6.7  42.1  12.59      0.696000     2
1        M2  0.91  0.83  7.0  31.7  10.11      0.940389     1
2        M3  0.79  0.62  4.8  46.7  13.23      0.457583     6
3        M4  0.78  0.61  6.4  42.4  12.55      0.607501     3
4        M5  0.94  0.88  3.6  62.2  16.91      0.400354     7


In [8]:
!pip install pyxlsx pandas